In [2]:
%pwd

'/mnt/d/gitfile/AFM-RepairTool'

In [3]:
import json
import os
from exp_process_funcs_block import *
from src.poscar import poscar
import einops
dataset = "../data/middle"
model_name = "unet_tune_v0"
use_poscar = False
dics = os.listdir(f"{dataset}/combine_afm")
#dics = ["pls2"]
#dics = ["HDA3", "HDA4", "HDA5", "HDA6", "HDA7", "HDA8", "ss0", "exp91"]
# dics = ["exp91"] 
# readme = json.load(open(f"{dataset}/combine_afm/{dic}/readme.json"))
# N_exp = readme["imgs"]
# L_exp, L_step, L_tar, stride = read_slide_info(f"{dataset}/afm/{dic}_cut_info.txt")
# L_exp

In [4]:
os.makedirs(f"{dataset}/combine", exist_ok=True)
os.makedirs(f"{dataset}/combine/{model_name}", exist_ok=True)

In [5]:
for dic in dics:
    readme = json.load(open(f"{dataset}/combine_afm/{dic}/readme.json"))
    N_exp = readme["imgs"]
    L_exp, L_step, L_tar, stride = read_slide_info(f"{dataset}/afm/{dic}_cut_info.txt")

    ldirs0 = os.listdir(f"{dataset}/afm")
    regex = re.compile(f"{dic}_" + r'(-?\d+)_(-?\d+)_(-?\d+)_(-?\d+)')
    ldirs = [i for i in ldirs0 if regex.match(i)]
    flag_list = [np.array(l.split('_')[1:]).astype(int) for l in ldirs]
    file_list = [f"{dataset}/result/{model_name}/{l}.poscar" for l in ldirs]
    Cubic_out = combine_cell_pos(file_list,flag_list,L_exp,L_tar,stride, 2.5 / 32)
    path_prediction = f"{dataset}/combine/{model_name}/{dic}.poscar"

    Cubic_out = einops.rearrange(torch.from_numpy(Cubic_out), "H W D (E C)-> D H W E C", C = 4)[...,(3,2,0,1)]
    point_dict = poscar.box2pos(Cubic_out, real_size = (3.0, L_exp[1] * 10, L_exp[0] * 10), threshold= 0.5, sort = False)
    poscar.pos2poscar(f"{dataset}/combine/{model_name}/{dic}.poscar", point_dict, real_size=(3.0, L_exp[1] * 10, L_exp[0] * 10))
    
    del Cubic_out
    
    file_list_npy = [f"{dataset}/npy/{model_name}/{l}.npy" for l in ldirs]
    Cubic_out_npy = combine_cell(file_list_npy,flag_list,L_exp,L_tar,stride, 2.5 / 32)
    path_prediction_npy = f"{dataset}/combine/{model_name}/{dic}.npy"

    Cubic_out_npy = einops.rearrange(torch.from_numpy(Cubic_out_npy), "H W D (E C)-> D H W E C", C = 4)[...,(3,2,0,1)]
    np.save(f"{dataset}/combine/{model_name}/{dic}.npy", Cubic_out_npy)
    point_dict = poscar.box2pos(Cubic_out_npy, real_size = (3.0, L_exp[1] * 10, L_exp[0] * 10), threshold= 0.5, sort = True)
    poscar.pos2poscar(f"{dataset}/combine/{model_name}/{dic}_npy.poscar", point_dict, real_size=(3.0, L_exp[1] * 10, L_exp[0] * 10))

    # positions_pred = target2positions(Cubic_out, info, ['O', 'H'],0.63, True)
    # positions_pred = target2positions_O(Cubic_out, info, ['O', 'H'],0.63, False) # use the restrict rules O:H 1:2
    # positions2poscar(positions_pred, info, path_prediction)

    # positions_pred_npy = target2positions_O(Cubic_out_npy, info, ['O', 'H'],0.63, True) # use the restrict rules O:H 1:2
    # positions2poscar(positions_pred_npy, info, path_prediction_npy)

[52 52]
[52 39]
[56 41]
[39 49]
[52 52]
[36 71]
[90 90]
[52 61]
